In [2]:
import difflib
import glob
from pprint import pprint
from pathlib import Path
import string
import random

import pandas as pd
from argon2 import PasswordHasher
from faker import Faker

from config import CONFIG
from utils import Utils

In [3]:
DATASET_PATH = str(("datasets/*/*/*/*/*"))
datasets = list(map(lambda x: Path(x), glob.glob(DATASET_PATH)))

pprint(f"Datasets:{datasets}")

("Datasets:[PosixPath('datasets/aprabowo/indonesia-tourism-destination/versions/1/package_tourism.csv'), "
 "PosixPath('datasets/aprabowo/indonesia-tourism-destination/versions/1/tourism_rating.csv'), "
 "PosixPath('datasets/aprabowo/indonesia-tourism-destination/versions/1/user.csv'), "
 "PosixPath('datasets/aprabowo/indonesia-tourism-destination/versions/1/tourism_with_id.csv')]")


In [4]:
df = {data_path.stem: pd.read_csv(data_path) for data_path in datasets}
print(df.keys())

dict_keys(['package_tourism', 'tourism_rating', 'user', 'tourism_with_id'])


## User Data

In [5]:
df["user"].head()

,User_Id,Location,Age
0,1,"Semarang, Jawa Tengah",20
1,2,"Bekasi, Jawa Barat",21
2,3,"Cirebon, Jawa Barat",23
3,4,"Bekasi, Jawa Barat",21
4,5,"Lampung, Sumatera Selatan",20


In [6]:
df["user"]["dob"] = df["user"]["Age"].apply(
    lambda age: Utils.calculate_dob_from_age(age)
)
df["user"].head()

,User_Id,Location,Age,dob
0,1,"Semarang, Jawa Tengah",20,2006-01-12
1,2,"Bekasi, Jawa Barat",21,2005-04-22
2,3,"Cirebon, Jawa Barat",23,2003-04-14
3,4,"Bekasi, Jawa Barat",21,2005-05-25
4,5,"Lampung, Sumatera Selatan",20,2006-01-03


In [7]:
df["user"]["Province"] = df["user"]["Location"].apply(
    lambda x: x.split(",")[1].strip() if pd.notnull(x) else x
)
df["user"].head()

,User_Id,Location,Age,dob,Province
0,1,"Semarang, Jawa Tengah",20,2006-01-12,Jawa Tengah
1,2,"Bekasi, Jawa Barat",21,2005-04-22,Jawa Barat
2,3,"Cirebon, Jawa Barat",23,2003-04-14,Jawa Barat
3,4,"Bekasi, Jawa Barat",21,2005-05-25,Jawa Barat
4,5,"Lampung, Sumatera Selatan",20,2006-01-03,Sumatera Selatan


In [8]:
password = CONFIG.USER_STARTER_PASSWORD.get_secret_value()  # type: ignore


In [9]:
ph = PasswordHasher()

In [10]:
hashed = ph.hash(password)

In [11]:
ph.verify(hashed, password)

True

In [12]:
df["user"]["password"] = hashed

In [13]:
fake = Faker("id_ID")

In [14]:
df["user"]["full_name"] = df["user"].apply(lambda row: fake.name(), axis=1)
df["user"]["username"] = df["user"]["full_name"].apply(
    lambda name: name[:10]
    .lower()
    .replace(" ", "_")
    .translate(str.maketrans("", "", string.punctuation))
    + str(random.randint(10, 99))
)
df["user"].head()

,User_Id,Location,Age,dob,Province,password,full_name,username
0,1,"Semarang, Jawa Tengah",20,2006-01-12,Jawa Tengah,"$argon2id$v=19$m=65536,t=3,p=4$xsNQzcNm8A3zz71...","KH. Harsanto Safitri, S.I.Kom",khharsan88
1,2,"Bekasi, Jawa Barat",21,2005-04-22,Jawa Barat,"$argon2id$v=19$m=65536,t=3,p=4$xsNQzcNm8A3zz71...",Vera Rajasa,verarajas93
2,3,"Cirebon, Jawa Barat",23,2003-04-14,Jawa Barat,"$argon2id$v=19$m=65536,t=3,p=4$xsNQzcNm8A3zz71...","H. Satya Nasyidah, S.Pd",hsatyan23
3,4,"Bekasi, Jawa Barat",21,2005-05-25,Jawa Barat,"$argon2id$v=19$m=65536,t=3,p=4$xsNQzcNm8A3zz71...",Shania Mahendra,shaniamah11
4,5,"Lampung, Sumatera Selatan",20,2006-01-03,Sumatera Selatan,"$argon2id$v=19$m=65536,t=3,p=4$xsNQzcNm8A3zz71...",Diah Adriansyah,diahadria41


In [15]:
user_insert_sql = """
insert into public.users (
  user_id,
  username,
  full_name,
  password,
  dob,
  province
) values
"""

In [16]:
len(df["user"].drop_duplicates()), len(df["user"])

(300, 300)

In [17]:
rows = []
for _, row in df["user"].drop_duplicates().iterrows():
    rows.append(
        f"({row['User_Id']}, '{row['username']}', '{row['full_name']}', '{row['password']}', '{row['dob']}', '{row['Province']}')"
    )
values_str = ",\n".join(rows)

In [18]:
with open("supabase/seeder/user.sql", "w") as f:
    f.write(user_insert_sql + values_str + ";")

## Tourism Place

In [19]:
df["tourism_with_id"].head()

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Unnamed: 11,Unnamed: 12
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,4.6,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153,NaN,1
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,4.6,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125,NaN,2
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000,4.6,360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538,NaN,3
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000,4.5,NaN,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-6.302446,106.895156,NaN,4
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000,4.5,60.0,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,106.839134,NaN,5


In [20]:
df["tourism_with_id"].columns

Index(['Place_Id', 'Place_Name', 'Description', 'Category', 'City', 'Price',
       'Rating', 'Time_Minutes', 'Coordinate', 'Lat', 'Long', 'Unnamed: 11',
       'Unnamed: 12'],
      dtype='object')

In [21]:
df["tourism_with_id"]["City"].value_counts()

City
Yogyakarta    126
Bandung       124
Jakarta        84
Semarang       57
Surabaya       46
Name: count, dtype: int64

In [22]:
df['tourism_with_id'].at[210, 'Place_Name'] = 'Gunung Tangkuban Perahu'



In [23]:
city_province_mapping = {
    "Jakarta": "Jakarta",
    "Bandung": "Jawa Barat",
    "Yogyakarta": "Yogyakarta",
    "Surabaya": "Jawa Timur",
    "Semarang": "Jawa Tengah",
}

In [24]:
df["tourism_with_id"]["Province"] = df["tourism_with_id"]["City"].apply(
    lambda city: city_province_mapping.get(city, "Unknown")
)

In [25]:
len(df["tourism_with_id"].drop_duplicates()), len(df["tourism_with_id"])

(437, 437)

In [27]:
place_insert_sql = """
insert into public.place (
  place_id,
  place_name,
  description,
  category,
  province,
  price,
  rating,
  time_minutes,
  latitude,
  longitude
) values
"""


def escape_sql(value):
    if isinstance(value, str):
        return value.replace("'", "''")
    return value


rows = []
for _, row in df["tourism_with_id"].iterrows():
    rows.append(
        f"({row['Place_Id']}, "
        f"'{escape_sql(row['Place_Name'])}', "
        f"'{escape_sql(row['Description'])}', "
        f"'{escape_sql(row['Category'])}', "
        f"'{escape_sql(row['City'])}', "
        f"{row['Price'] if pd.notnull(row['Price']) else 'NULL'}, "
        f"{row['Rating'] if pd.notnull(row['Rating']) else 'NULL'}, "
        f"{row['Time_Minutes'] if pd.notnull(row['Time_Minutes']) else 'NULL'}, "
        f"{row['Lat']}, "
        f"{row['Long']}"
        ")"
    )
values_str = ",\n".join(rows)

with open("supabase/seeder/place.sql", "w") as f:
    f.write(place_insert_sql + values_str + ";")

## User Place Interaction

In [28]:
df["tourism_rating"].head()

,User_Id,Place_Id,Place_Ratings
0,1,179,3
1,1,344,2
2,1,5,5
3,1,373,3
4,1,101,4


In [37]:
len(df["tourism_rating"].drop_duplicates()), len(df["tourism_rating"])

(9921, 9921)

In [30]:
df["tourism_rating"].drop_duplicates(inplace=True)

In [36]:
df['tourism_rating'][df["tourism_rating"].duplicated()]

,User_Id,Place_Id,Place_Ratings


In [38]:
user_place_rating_insert_sql = """
insert into public.user_place_rating (
    user_id,
    place_id,
    rating
) values
"""
rows = []
for _, row in df["tourism_rating"].iterrows():
    rows.append(
        f"({row['User_Id']}, "
        f"{escape_sql(row['Place_Id'])},"
        f"{escape_sql(row['Place_Ratings'])}"
        ")"
    )
values_str = ",\n".join(rows)

with open("supabase/seeder/user_place_rating.sql", "w") as f:
    f.write(user_place_rating_insert_sql + values_str + ";")

## Place Package

In [30]:
df['package_tourism'].head()

,Package,City,Place_Tourism1,Place_Tourism2,Place_Tourism3,Place_Tourism4,Place_Tourism5
0,1,Jakarta,Pasar Tanah Abang,Taman Ayodya,Museum Tekstil,NaN,NaN
1,2,Jakarta,Pasar Tanah Abang,Pasar Taman Puring,Pasar Petak Sembilan,NaN,NaN
2,3,Jakarta,Perpustakaan Nasional,Monas,Masjid Istiqlal,NaN,NaN
3,4,Jakarta,Pulau Tidung,Pulau Bidadari,Pulau Pari,Pulau Pramuka,Pulau Pelangi
4,5,Jakarta,Museum Satria Mandala,Museum Wayang,Museum Bahari Jakarta,Museum Macan (Modern and Contemporary Art in N...,NaN


In [31]:
len(df['package_tourism'])

100

In [32]:
flat_df = pd.melt(
    df['package_tourism'],
    id_vars=['Package', 'City'],
    value_vars=[
        'Place_Tourism1', 'Place_Tourism2', 'Place_Tourism3',
        'Place_Tourism4', 'Place_Tourism5'
    ],
    var_name='place_tourism',
    value_name='place_name'
).dropna(subset=['place_name'])

flat_df = flat_df.rename(columns={'Package': 'package', 'City': 'city'})

flat_df = flat_df[['package', 'city', 'place_name']]

In [33]:
def name_to_id(name:str, df_place:pd.DataFrame = df['tourism_with_id']):
    """Convert place name to its corresponding ID."""
    place_id = df_place.query(f"Place_Name == '{name}'")['Place_Id'].values
    return place_id[0] if len(place_id) > 0 else None


In [34]:

flat_df['place_id'] = flat_df['place_name'].apply(name_to_id)

In [35]:
flat_df.head()

,package,city,place_name,place_id
0,1,Jakarta,Pasar Tanah Abang,25.0
1,2,Jakarta,Pasar Tanah Abang,25.0
2,3,Jakarta,Perpustakaan Nasional,70.0
3,4,Jakarta,Pulau Tidung,10.0
4,5,Jakarta,Museum Satria Mandala,82.0


In [36]:
flat_df[flat_df['place_id'].isna()]

,package,city,place_name,place_id
102,3,Jakarta,Monas,NaN
130,31,Yogyakarta,Taman Pelangi Jogja,NaN
207,8,Jakarta,Monas,NaN
248,49,Bandung,Panama Park 825,NaN
262,63,Semarang,Pantai Cipta,NaN
267,68,Semarang,Semarang Gallery,NaN
286,87,Surabaya,| Food Junction Grand Pakuwon,NaN
431,32,Yogyakarta,Wisata Kalibiru,NaN
449,50,Bandung,| Peta Park,NaN
451,52,Bandung,Stone Garden Geopark,NaN


In [37]:
flat_df["place_name"] = flat_df["place_name"].map(lambda x: x.replace("|", "").strip())

In [38]:

flat_df['place_id'] = flat_df['place_name'].apply(name_to_id)

In [39]:
flat_df[flat_df['place_id'].isna()]

,package,city,place_name,place_id
102,3,Jakarta,Monas,NaN
130,31,Yogyakarta,Taman Pelangi Jogja,NaN
207,8,Jakarta,Monas,NaN
267,68,Semarang,Semarang Gallery,NaN
431,32,Yogyakarta,Wisata Kalibiru,NaN
451,52,Bandung,Stone Garden Geopark,NaN
452,53,Bandung,Ade Irma Suryani Nasution Traffic Park,NaN


In [40]:
# Get all place names
place_names = df["tourism_with_id"]["Place_Name"].tolist()

# Find close matches
similar = difflib.get_close_matches("Gunung Tangkuban Perahu", place_names, n=5, cutoff=0.5)
print("Similar place names:", similar)

Similar place names: ['Gunung Tangkuban Perahu', 'Gunung Manglayang', 'Watu Gunung Ungaran', 'Gunung Papandayan', 'Gunung Lalakon']


In [137]:
#Fix gunung tangkuban perahu
df["tourism_with_id"].query("Place_Name == 'GunungTangkuban perahu'") 

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Unnamed: 11,Unnamed: 12,Province


In [41]:
df['tourism_with_id'].at[210, 'Place_Name'] = 'Gunung Tangkuban Perahu'


In [42]:
flat_df['place_id'] = flat_df['place_name'].apply(name_to_id)

In [43]:
flat_df[flat_df['place_id'].isna()]

,package,city,place_name,place_id
102,3,Jakarta,Monas,NaN
130,31,Yogyakarta,Taman Pelangi Jogja,NaN
207,8,Jakarta,Monas,NaN
267,68,Semarang,Semarang Gallery,NaN
431,32,Yogyakarta,Wisata Kalibiru,NaN
451,52,Bandung,Stone Garden Geopark,NaN
452,53,Bandung,Ade Irma Suryani Nasution Traffic Park,NaN


In [44]:
similar = difflib.get_close_matches("Monumen", place_names, n=5, cutoff=0.5)
print("Similar place names:", similar)

Similar place names: ['Monumen Sanapati', 'Monumen Nasional', 'Monumen Kapal Selam', 'Monumen Yogya Kembali', 'Monumen Tugu Pahlawan']


In [45]:
flat_df.at[102, 'place_name'] = 'Monumen Nasional'
flat_df.at[207, 'place_name'] = 'Monumen Nasional'
flat_df['place_id'] = flat_df['place_name'].apply(name_to_id)
flat_df[flat_df['place_id'].isna()]

,package,city,place_name,place_id
130,31,Yogyakarta,Taman Pelangi Jogja,NaN
267,68,Semarang,Semarang Gallery,NaN
431,32,Yogyakarta,Wisata Kalibiru,NaN
451,52,Bandung,Stone Garden Geopark,NaN
452,53,Bandung,Ade Irma Suryani Nasution Traffic Park,NaN


In [46]:
name_to_id("Taman Pelangi Yogyakarta")

np.int64(98)

In [47]:
similar = difflib.get_close_matches("Taman Pelangi Jogja", place_names, n=5, cutoff=0.5)
print("Similar place names:", similar)
flat_df.at[130, 'place_name'] = 'Taman Pelangi Yogyakarta'
flat_df['place_id'] = flat_df['place_name'].apply(name_to_id)
flat_df[flat_df['place_id'].isna()]

Similar place names: ['Taman Pelangi', 'Taman Pelangi Yogyakarta', 'Kampung Pelangi', 'Taman Lansia', 'Taman Begonia']


,package,city,place_name,place_id
267,68,Semarang,Semarang Gallery,NaN
431,32,Yogyakarta,Wisata Kalibiru,NaN
451,52,Bandung,Stone Garden Geopark,NaN
452,53,Bandung,Ade Irma Suryani Nasution Traffic Park,NaN


In [48]:
similar = difflib.get_close_matches("Semarang Gallery", place_names, n=5, cutoff=0.5)
print("Similar place names:", similar)
flat_df.at[267, 'place_name'] = 'Semarang Contemporary Art Gallery'
flat_df['place_id'] = flat_df['place_name'].apply(name_to_id)
flat_df[flat_df['place_id'].isna()]

Similar place names: ['Semarang Contemporary Art Gallery', 'Semarang Chinatown', 'Taman Cattleya', 'Jembatan Merah']


,package,city,place_name,place_id
431,32,Yogyakarta,Wisata Kalibiru,NaN
451,52,Bandung,Stone Garden Geopark,NaN
452,53,Bandung,Ade Irma Suryani Nasution Traffic Park,NaN


In [49]:
similar = difflib.get_close_matches("Wisata Kalibiru", place_names, n=5, cutoff=0.5)
print("Similar place names:", similar)
flat_df.at[431, 'place_name'] = 'Wisata Alam Kalibiru'
flat_df['place_id'] = flat_df['place_name'].apply(name_to_id)
flat_df[flat_df['place_id'].isna()]

Similar place names: ['Wisata Alam Kalibiru', 'Wisata Kaliurang', 'Wisata Lereng Kelir', 'Desa Wisata Kelor', 'Wisata Eling Bening']


,package,city,place_name,place_id
451,52,Bandung,Stone Garden Geopark,NaN
452,53,Bandung,Ade Irma Suryani Nasution Traffic Park,NaN


In [50]:
similar = difflib.get_close_matches("Stone Garden Geopark", place_names, n=5, cutoff=0.5)
print("Similar place names:", similar)
flat_df.at[451, 'place_name'] = 'Stone Garden Citatah'
flat_df['place_id'] = flat_df['place_name'].apply(name_to_id)
flat_df[flat_df['place_id'].isna()]

Similar place names: ['Stone Garden Citatah', 'Ocean Ecopark', 'Tektona Waterpark']


,package,city,place_name,place_id
452,53,Bandung,Ade Irma Suryani Nasution Traffic Park,NaN


In [51]:
similar = difflib.get_close_matches("Ade Irma Suryani Nasution Traffic Park", place_names, n=5, cutoff=0.5)
print("Similar place names:", similar)
flat_df.at[452, 'place_name'] = 'Taman Lalu Lintas Ade Irma Suryani Nasution'
flat_df['place_id'] = flat_df['place_name'].apply(name_to_id)
flat_df[flat_df['place_id'].isna()]

Similar place names: ['Taman Lalu Lintas Ade Irma Suryani Nasution']


,package,city,place_name,place_id


In [52]:
len(flat_df.drop_duplicates()), len(flat_df)

(403, 405)

In [53]:
tour_package_insert_sql = """
insert into public.tour_package (
    tour_package_id,
    place_id
) values 
"""
rows = []
for _, row in flat_df.drop_duplicates().iterrows():
    rows.append(
        f"({row['package']}, "
        f"{row['place_id']}"
        ")"
    )
values_str = ",\n".join(rows)

with open("supabase/seeder/tour_package.sql", "w") as f:
    f.write(tour_package_insert_sql + values_str + ";")
